In [ ]:
# Set up key libraries / tokens

In [1]:
import requests 
from requests_oauthlib import OAuth1
import os
import pandas as pd

In [2]:
API_PUBLIC = os.environ.get("TW_API_KEY")
API_SECRET = os.environ.get("TW_API_SECRET_KEY")
APP_PUBLIC = os.environ.get("TW_PRS_GA_HW_ACCESS_TOKEN")
APP_SECRET = os.environ.get("TW_PRS_GA_HW_ACCESS_TOKEN_SECRET")

In [16]:
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
auth = OAuth1(
            API_PUBLIC, #'USER_OAUTH_TOKEN', 
            API_SECRET, #'USER_OAUTH_TOKEN_SECRET'
            APP_PUBLIC, #'YOUR_APP_KEY', 
            APP_SECRET #'YOUR_APP_SECRET',
            )

In [ ]:
# Check authorisation is working properly

In [98]:
check_auth = requests.get(url, auth=auth)

In [99]:
check_auth.reason

'OK'

In [101]:
check_auth.status_code

200

# Function 3

**Name:** `get_followers`

**Returns:** list of data objects for each of the users followers, returning values for the `name`, `followers_count`, `friends_count`, and `screen_name` key for each user.

**Arguments:** 

 - `screen_name`: str, required; Twitter handle to search for.  **Results should not depend on user inputting the @ symbol.**
 - `keys`: list, required;  keys to return for each user.  default value: [`name`, `followers_count`, `friends_count`, `screen_name`]; if something else is listed, values for those keys should be returned
 - `to_df`: bool, required; default value: False; if True, return results in a dataframe.  Every value provided in the `keys` argument should be its own column, with rows populated by the corresponding values for each one for every user.
 
**To Test:** We'll test your functions in the following ways:

 - `get_followers('@GA')`
 - `get_followers('GA')`
 - `get_followers('GA', keys=['name', 'followers_count'])`
 - `get_followers('GA', keys=['name', 'followers_count'], to_df=True)`
 - `get_followers('GA', to_df=True)`

In [102]:
def get_followers(
        screen_name,
        keys = [
                'name',
                'followers_count',
                'friends_count',
                'screen_name'
                ], 
        to_df = False,
        auth=auth
        ):

    # Checks on screen_name input
    if screen_name[0] == '@':
        screen_name = screen_name[1:]

    # Go and get results, adding users to an overall list
    user_list = []
    cursor = -1
    while cursor != 0:
        find_follower_url = ("https://api.twitter.com/1.1/followers/list.json" 
                         + "?screen_name=" + screen_name
                         + "&cursor=" + str(cursor)
                         + "&count=200" 
                        )
        #print(f'find_user_url: {find_user_url}')
        follower_output = requests.get(find_follower_url, auth=auth)
        if follower_output.status_code == 429:
            print("Reached max API requests.\n"
                  + "Setting cursor to 0 to exit loop.\n"
                  + "Followers found so far will be output."
                 )
            cursor = 0
        else:
            follower_output_dict = follower_output.json()
            cursor = follower_output_dict['next_cursor']
            [user_list.append(user) for user in follower_output_dict['users']]
        
    # Limit the user_list to the specific keys required
    user_list_keys = []
    for user in user_list:
        temp_dict = {}
        for k in keys:
            temp_dict.update({k:user[k]})
        user_list_keys.append(temp_dict)
    
    # Allow for pandas data frame
    if to_df == False:
        return_object = user_list_keys
    elif to_df == True:
        # Loop through each potential column, and add to dictionary
        prep_df_dict = {}
        
        for k in keys:
            temp_list = [i[k] for i in user_list_keys]
            prep_df_dict.update({k:temp_list})
        
        #Add data to the data frame
        user_df = pd.DataFrame(data=prep_df_dict)
        
        return_object = user_df
    else:
        return_object = 'to_df argument only accpets True or False'
                
    return return_object


In [103]:
otest = get_followers('@ickbat',to_df=True)

In [104]:
len(otest)

586

In [105]:
otest

,name,followers_count,friends_count,screen_name
0,"omi chun (xe/him), ""machiavellian status-chaser""",463,2388,omi_the_fool
1,AJ,1,62,AJ46802226
2,Sam Tung,481,2171,stung_art
3,in🎲die zine,161,32,in_die_zine
4,Rob Goddard,180,391,rgoddard_
...,...,...,...,...
581,Jen Breach,425,82,jenbreach
582,Brennan Taylor,1841,541,brennanrtaylor
583,Privateer Press,24453,15735,privateerpress
584,SteampunkPoland,134,134,SteampunkPoland


In [106]:
otest['followers_count'].sum()

2390389

In [83]:
me = [i for i in otest if i['screen_name'] == 'prsinkis']

In [84]:
me

[{'name': 'Peter Sinkis',
  'followers_count': 15,
  'friends_count': 80,
  'screen_name': 'prsinkis'}]

In [90]:
user_id = otest['user_1']['id']
follower_list_url = 'https://api.twitter.com/1.1/followers/list.json?screen_name=' + 'prsinkis'
follower_list_url = 'https://api.twitter.com/1.1/followers/list.json?screen_name=prsinkis&count=200&cursor=-1'
follower_output = requests.get(follower_list_url, auth=auth)

KeyError: 'user_1'

In [ ]:
follower_output

In [ ]:
follower_output_dict = follower_output.json()

In [ ]:
for k in follower_output_dict.keys():
    print(k)

In [ ]:
len(follower_output_dict['users'])

In [ ]:
for item in follower_output_dict['users']:
    print(item['name'])

In [ ]:
for k in follower_output_dict['users'][0].keys():
    print(k)

In [ ]:
print(follower_output_dict['total_count'])

In [ ]:
print(follower_output_dict['next_cursor'])

In [ ]:
print(follower_output_dict)

In [ ]:
a = [1,2,3]
b = [4,5,6]
print(a)
print(b)
[a.append(i) for i in b]
print(a)

In [15]:
otest.json()


{'errors': [{'code': 32, 'message': 'Could not authenticate you.'}]}

In [48]:
'abc' + str(-1) + 'def'

'abc-1def'

NameError: name 'to_df' is not defined

In [ ]:
test1 = get_followers('@GA')
test2 = get_followers('GA')
test3 = get_followers('GA', keys=['name', 'followers_count'])
test4 = get_followers('GA', keys=['name', 'followers_count'], to_df=True)
test5 = get_followers('GA', to_df=True)

In [107]:
test4 = get_followers('GA', keys=['name', 'followers_count'], to_df=True)

Reached max API requests. Setting cursor to 0 to exit loop


In [108]:
test4

,name,followers_count
0,🎃 Blumpy,464
1,Atiqur Rahman Sourav,27
2,Dalyah Aljamal,103
3,Jennifer Schneider,301
4,MFW,55
...,...,...
2395,Bela Fitri Aulia,83
2396,ナディア,18
2397,Magdalena Moursy,72
2398,Ayula,1932
